In [1]:
# ============================================================
# 🏅 Subtitulador automático con Hugging Face + Whisper + Gradio
# ============================================================

!pip install -q transformers gradio datasets moviepy ffmpeg-python huggingface_hub pysrt



In [2]:
# ==============================================
# 📦 Importaciones principales del proyecto
# ==============================================

# 🔇 Silenciar warnings de MoviePy (no afectan al código)
import warnings
warnings.filterwarnings("ignore", category=SyntaxWarning)

# 🚀 Librerías necesarias
from transformers import pipeline
import gradio as gr
from moviepy.editor import VideoFileClip
import tempfile, os, datetime, ffmpeg, pysrt


In [3]:
# ------------------------------------------------------------
# 🔧 Generar un archivo .srt usando los tiempos reales de Whisper
# ------------------------------------------------------------
def generar_srt_desde_chunks(chunks, srt_path):
    subs = pysrt.SubRipFile()

    for i, chunk in enumerate(chunks, 1):
        start_time = chunk["timestamp"][0] or 0
        end_time = chunk["timestamp"][1] or (start_time + 3)

        def td_to_srt_time(seconds):
            total_seconds = int(seconds)
            hours = total_seconds // 3600
            minutes = (total_seconds % 3600) // 60
            secs = total_seconds % 60
            milliseconds = int((seconds - total_seconds) * 1000)
            return pysrt.SubRipTime(hours=hours, minutes=minutes, seconds=secs, milliseconds=milliseconds)

        # Limit subtitle text to 8 words per line
        words = chunk["text"].strip().split()
        lines = []
        current_line_words = []
        for word in words:
            current_line_words.append(word)
            if len(current_line_words) >= 8:
                lines.append(" ".join(current_line_words))
                current_line_words = []
        if current_line_words:
            lines.append(" ".join(current_line_words))

        formatted_text = "\n".join(lines)

        subs.append(pysrt.SubRipItem(
            index=i,
            start=td_to_srt_time(start_time),
            end=td_to_srt_time(end_time),
            text=formatted_text
        ))

    subs.save(srt_path, encoding='utf-8')


In [4]:
# ------------------------------------------------------------
# 🎬 Procesa el video: usa Whisper con timestamps reales
# ------------------------------------------------------------
def subtitular_y_exportar(video_path):
    try:
        # Extraer audio temporal
        with tempfile.NamedTemporaryFile(suffix=".wav", delete=False) as temp_audio:
            clip = VideoFileClip(video_path)
            clip.audio.write_audiofile(temp_audio.name, codec="pcm_s16le", verbose=False, logger=None)
            clip.close()
            audio_path = temp_audio.name

        # Modelo Whisper (detecta idioma y tiempos)
        asr = pipeline("automatic-speech-recognition", model="openai/whisper-small", return_timestamps=True)

        # Transcribir audio con tiempos
        result = asr(audio_path)
        texto = result["text"]

        # Crear subtítulos reales con timestamps
        srt_path = "subtitulos.srt"
        generar_srt_desde_chunks(result["chunks"], srt_path)

        os.remove(audio_path)

        # Crear video con subtítulos precisos
        output_path = "video_subtitulado.mp4"
        (
            ffmpeg
            .input(video_path)
            .output(
                output_path,
                vf=f"subtitles={srt_path}:force_style='FontName=Arial,FontSize=24,PrimaryColour=&HFFFFFF&,"
                   f"OutlineColour=&H000000&,BorderStyle=3,BackColour=&H80000000&,Alignment=2'",
                vcodec='libx264', acodec='aac', strict='experimental'
            )
            .overwrite_output()
            .run(quiet=True)
        )

        return output_path, srt_path, texto

    except Exception as e:
        return None, None, f"⚠️ Error procesando el video: {str(e)}"


In [5]:
# ------------------------------------------------------------
# 🧠 Generar resumen del texto transcrito
# ------------------------------------------------------------

def resumir_texto(texto):
    try:
        summarizer = pipeline("summarization", model="csebuetnlp/mT5_multilingual_XLSum")
        resumen = summarizer(texto, max_length=130, min_length=30, do_sample=False)[0]["summary_text"]
        return resumen
    except Exception as e:
        return f"⚠️ Error al generar resumen: {str(e)}"

# ------------------------------------------------------------
# 🧩 Función que usa subtitulador y añade el resumen
# ------------------------------------------------------------
def ejecutar_con_resumen(video):
    video_sub, srt_file, subtitulos = ejecutar(video)  # usa la función original
    resumen = resumir_texto(subtitulos)
    return video_sub, srt_file, subtitulos, resumen


In [6]:
# ------------------------------------------------------------
# 💡 Función que Gradio ejecuta
# ------------------------------------------------------------
def ejecutar(video):
    video_sub, srt_file, subtitulos = subtitular_y_exportar(video)
    return video_sub, srt_file, subtitulos


In [7]:
# ------------------------------------------------------------
# 🌐 Interfaz de usuario con Gradio
# ------------------------------------------------------------
demo_resumen = gr.Interface(
    fn=ejecutar_con_resumen,
    inputs=gr.Video(label="🎥 Sube tu video"),
    outputs=[
        gr.Video(label="✅ Video subtitulado"),
        gr.File(label="⬇️ Archivo .SRT"),
        gr.Textbox(label="📝 Texto transcrito", lines=5),
        gr.Textbox(label="🧠 Resumen del video", lines=3)
    ],
    title="🏅 Subtitulador Automático + Resumen (Whisper + Hugging Face)",
    description="Sube un video, obtén los subtítulos, el texto y un resumen automático."
)

demo_resumen.launch(share=True)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://aa9c634128d28ed5f2.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [8]:

# Opcional: borrar archivos temporales creados en ejecuciones anteriores
!rm -f subtitulos.srt video_subtitulado.mp4